In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
raw_dat = pd.read_excel("../data/raw_data.xlsx")
raw_dat['Normalized'] = (raw_dat['mkt_r'] - raw_dat['dep_f_r'])/(raw_dat['len_f_r'] - raw_dat['dep_f_r'])
dat = raw_dat.drop(columns=['mkt_r', 'dep_f_r', 'len_f_r'])
dat.to_csv("../data/reserve_demand_cruve_data.csv", index=False)

# Non-Parametric Modeling

In [3]:
import reserves_demand.curve_nparam_fit as npcf

In [4]:
obj = npcf.CurveNonParamFit(
    data = dat, 
    dep_var = 'Normalized',
    main_indep_var= 'excess_reserve',
    dep_var_name= 'Normalized Overnight Rate',                 
    main_indep_var_name= 'Excess Reserve',
    nFolds=5,
)

In [5]:
best_model_type, comparison_result  = obj.compare_models(verbose = True, plot=True)
comparison_result.set_index('Model Type').T

In [ ]:
obj.plot_best()

In [ ]:
forecast_input = dat.tail(2).drop(columns=['Normalized'])
forecast_output = obj.predict_best('Best Model', X=forecast_input)
forecast_output

# Parametric Modeling

In [ ]:
import reserves_demand.curve_param_fit as pcf
obj = pcf.CurveParamFit(
    data = dat, 
    dep_var = 'Normalized',
    main_indep_var= 'excess_reserve',
    dep_var_name= 'Normalized Overnight Rate',                 
    main_indep_var_name= 'Excess Reserve',
    constant = True,
    Q = 0.9,
    nFolds=5,
    parallel=True,
    search_method = 'all_combinations'
)

In [ ]:
_ = obj.plot_x_y()

In [ ]:
best_curve, cv_result = obj.variable_select(on='RMSE', verbose=True, plot=True)
cv_result.set_index('Curve').T

In [ ]:
obj.indep_vars_occurence(normalize=False)

In [ ]:
best_params = obj.fit_best_curves()

In [ ]:
perf_metrics_df, predictions_df, param_df = obj.compare_best_curves(CI = True)

In [ ]:
perf_metrics_df

In [ ]:
predictions_df.head()

In [ ]:
param_df.loc['logistic']

In [ ]:
forecast_input = dat.tail(2).drop(columns=['Normalized'])
forecast_output = obj.predict_best_curve('logistic', X=forecast_input)
forecast_output

In [ ]:
obj.most_probable_value(curvename='logistic', known_value=('excess_reserve', 1755207), verbose=True)

In [ ]:
obj.most_probable_value(curvename='logistic', known_value=('Normalized', 0.38), verbose=True)

# Parametric Additive Model

In [ ]:
import reserves_demand.curve_paramadd_fit as pcf
obj = pcf.CurveParamAdditiveFit(
    data = dat, 
    dep_var = 'Normalized',
    main_indep_var= 'excess_reserve',
    dep_var_name= 'Normalized Overnight Rate',                 
    main_indep_var_name= 'Excess Reserve',
    Q = 0.9,
    nFolds=5,
    parallel=True,
    search_method = 'all_combinations'
)

In [ ]:
best_curve, cv_result = obj.variable_select(on='RMSE', verbose=True, plot=True)
cv_result.set_index('Curve').T

In [ ]:
obj.indep_vars_occurence(normalize=False)

In [ ]:
best_params = obj.fit_best_curves()

In [ ]:
perf_metrics_df, predictions_df, param_df = obj.compare_best_curves(CI = True)

In [ ]:
perf_metrics_df

In [ ]:
forecast_input = dat.tail(2).drop(columns=['Normalized'])
forecast_output = obj.predict_best_curve('logistic', X=forecast_input)
forecast_output

In [ ]:
obj.most_probable_value(curvename='logistic', known_value=('excess_reserve', 1755207), verbose=True)

In [ ]:
obj.most_probable_value(curvename='logistic', known_value=('Normalized', 0.38), verbose=True)